In [1]:
# import pandas as pd
# import os
# import cv2


# df = pd.read_csv('training_labels.csv')
# for index, row in df.iterrows():
#   img = cv2.imread(f"training_data/{row['id']:06}.jpg")
#   if not os.path.exists(f"tmp/{row['label']}"):
#     os.makedirs(f"tmp/{row['label']}")
#     print(f"{row['label']}")
#   cv2.imwrite(f"tmp/{row['label']}/{row['id']:06}.jpg", img)
#     #os.rmdir(row['label'])
#   #print(f"{row['id']:06}")

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
import random
import PIL
from PIL import Image, ImageOps

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(device=gpu, enable=True)

len(os.listdir(f'tmp'))  # Directory where training data folders are

196

In [2]:
tf.config.experimental.list_physical_devices(device_type='GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

###  Hyperparameters

batch size 256: 0.798

batch size 128: 0.833

batch size 64: 0.858

batch size 32: 0.881

batch size 16 0.891

In [3]:
batch_size = 16  # Training batch size
num_classes = 196  # Classes in dataset
num_epochs = 100   # Epochs for training   
lr = 1e-3  # Learning rate
lr_weight_decay = 1e-3 # Learning weight decay


### Data Processing

In [4]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
# validation_datagen=ImageDataGenerator(rescale=1/255)

# validation_generator = validation_datagen.flow_from_directory('hw5_data/test/',target_size=(299,299),
#                                                  batch_size=batch_size,
#                                                  class_mode='sparse')
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

"""
from albumentations.augmentations.transforms import Rotate
from albumentations import Compose

def augment_and_show(image):
    #image = Equalize(mode='pil', always_apply=True, p=1)(image=np.array(image).astype('uint8'))['image']
    aug = Compose([Rotate(limit=5, interpolation=2, p=1)])
    image = aug(image=image)['image']
    return image.astype('float')
"""

# preprocessing image and divide validaiton set
train_datagen = ImageDataGenerator(horizontal_flip=True, validation_split=0)
val_datagen = ImageDataGenerator(horizontal_flip=False, validation_split=0)

train_generator = train_datagen.flow_from_directory('tmp/',
                                                 target_size=(256,256),
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                 subset='training')

# validation_datagen = ImageDataGenerator()

# validation_generator = validation_datagen.flow_from_directory('hw5_data/test/',target_size=(256,256),
#                                                  batch_size=batch_size,
#                                                  class_mode='sparse')
validation_generator = val_datagen.flow_from_directory('tmp_test/',
                                                 target_size=(256,256),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')


# for i in range(25):
#     plt.figure(figsize=(30,20))
#     plt.subplot(5,5,i+1)
#     plt.imshow(train_generator.next()[0][i].astype('uint8'))
#     plt.tight_layout()

Found 11185 images belonging to 196 classes.
Found 4958 images belonging to 196 classes.


In [5]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

from classification_models.tfkeras import Classifiers

ResNet50, preprocess_input = Classifiers.get('resnet50')
base_model = ResNet50(input_shape=(256, 256, 3), weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(base_model.output)
#output = Dense(num_classes, kernel_initializer='zeros')(x)
output = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=[base_model.input], outputs=[output])


# Training with the final custom-made layers
# ResNet 50 (48,51)
for layer in model.layers[:48]:
    layer.trainable =False
    
        
for layer in model.layers[48:]:
    layer.trainable=True
    
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 256, 256, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
____________________________________________________________________________________________

### Callbacks

In [7]:
import math
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras import backend as K


class CosineAnnealingScheduler(Callback):
    """Cosine annealing scheduler.
    """

    def __init__(self, T_max, eta_max, eta_min=0, verbose=1):
        super(CosineAnnealingScheduler, self).__init__()
        self.T_max = T_max
        self.eta_max = eta_max
        self.eta_min = eta_min
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = self.eta_min + (self.eta_max - self.eta_min) * (1 + math.cos(math.pi * epoch / self.T_max)) / 2
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nEpoch %05d: CosineAnnealingScheduler setting learning '
                  'rate to %s.' % (epoch + 1, lr))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)
        
cosineanneling = CosineAnnealingScheduler(T_max=20, eta_max=1e-2, eta_min=1e-5)


checkpoint = ModelCheckpoint('logs/ep{epoch:03}-val_loss{val_loss:.3f}-val_auc{val_accuracy}.h5', save_best_only=False, save_weight_only=True, monitor='val_loss', mode='min', verbose=1)
#checkpoint = ModelCheckpoint('logs/ep{epoch:03}-loss{loss:.3f}-auc{accuracy}.h5', save_best_only=False, save_weight_only=True, monitor='loss', mode='min', verbose=1)

In [ ]:
from tensorflow.keras import optimizers
# Use default adam optimizer (learning rate=1e-3, decay=0)
# Loss function: categorical cross entropy
# Evaluation: Accuracy

sgd = optimizers.SGD(lr=1e-3, momentum=0.9, decay=1e-3)
#loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])


step_size_train = train_generator.n // train_generator.batch_size
step_size_val = validation_generator.n // validation_generator.batch_size

history = model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train,
                              validation_data=validation_generator, validation_steps=step_size_val,
                              epochs=num_epochs, callbacks=[cosineanneling, checkpoint])

Instructions for updating:
Please use Model.fit, which supports generators.

Epoch 00001: CosineAnnealingScheduler setting learning rate to 0.01.
Epoch 1/100
322/699 [============>.................] - ETA: 2:35 - loss: 4.2168 - accuracy: 0.1267

### Multi-GPU

In [ ]:
# from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
# from tensorflow.keras.models import Model
# import efficientnet.tfkeras as efn 
# import tensorflow as tf
# from tensorflow.keras import optimizers

# def get_compiled_model():

#     base_model = efn.EfficientNetB0(input_shape=(256, 256, 3), weights='imagenet', include_top=False)  # or weights='noisy-student'
#     x = GlobalAveragePooling2D()(base_model.output)
#     output = Dense(num_classes, activation='softmax')(x)
#     model = Model(inputs=[base_model.input], outputs=[output])
    
# #     for layer in model.layers[:22]:
# #         layer.trainable =False

# #     for layer in model.layers[22:]:
# #         layer.trainable=True

#     sgd = optimizers.SGD(lr=1e-3, momentum=0.9, decay=1e-3)
#     model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    
#     return model

# strategy = tf.distribute.MirroredStrategy()
# print('Number of devices: {}'.format(strategy.num_replicas_in_sync))



# step_size_train = train_generator.n // train_generator.batch_size
# step_size_val = validation_generator.n // validation_generator.batch_size


# with strategy.scope():
#   # Everything that creates variables should be under the strategy scope.
#   # In general this is only model construction & `compile()`.
#     model = get_compiled_model()
   
    
# history = model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train,
#                               validation_data=validation_generator, validation_steps=step_size_val,
#                               epochs=num_epochs , callbacks=[cosineanneling, checkpoint])

### Visualization

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('ResNet50 Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('ResNet50 Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

### Testing

In [9]:
model.load_weights(f'logs/ResNet_batch16_0.87_val_auc0.958.h5')
#model.predict(test_img)

In [10]:
import pandas as pd

class_label =  os.listdir(f'testing_data/')

tmp_dic = train_generator.class_indices
dic = dict((v,k) for k,v in tmp_dic.items())
dic[173] = 'Ram C/V Cargo Van Minivan 2012'

def DataLoader(standardize=False):
    img_list = []
    label_list = []
    filenames = []
    num_img = 0
    for img in os.listdir(f'testing_data/'):
        if img.endswith('.jpg'):
            original = cv2.imread(f'testing_data/{img}')
            filenames.append(img.split('.')[0])

            #original = augment_and_show(original)
            resized_img = cv2.resize(original, (256, 256), interpolation=cv2.INTER_CUBIC)

            img_list.append(resized_img)
            num_img += 1
    #img_list = img_list.T[1:,:]
    return np.array(img_list), filenames, num_img


test_img, name, num_test = DataLoader(standardize=False)
predict = (model.predict(test_img)).argmax(-1)




df = pd.DataFrame({
    'id': name,
    'label': [dic[num] for num in predict]
})

# dic = train_generator.class_indices
# label_num = []

# for label in test_label:
#     label_num.append(dic[label])

In [11]:
df.head()

,id,label
0,011838,Jaguar XK XKR 2012
1,015736,Volkswagen Golf Hatchback 1991
2,015586,Volkswagen Golf Hatchback 2012
3,009699,GMC Terrain SUV 2012
4,007597,Dodge Challenger SRT8 2011


In [13]:
df.to_csv("hw1_resnet50.csv", header=["id", "label"], index=False)

In [8]:
class_label =  os.listdir(f'tmp_test/')

def DataLoader(path, standardize=False):
    img_list = []
    label_list = []
    num_img = 0
    for classes in class_label:
        for img in os.listdir(f'tmp_test/{classes}'):
            if img.endswith('.jpg'):
                label_list.append(classes)
                original = cv2.imread(f'tmp_test/{classes}/{img}')
                

                #original = augment_and_show(original)
                resized_img = cv2.resize(original, (256, 256), interpolation=cv2.INTER_CUBIC)

                img_list.append(resized_img)
                num_img += 1
    #img_list = img_list.T[1:,:]
    return np.array(img_list), label_list, num_img


test_img, test_label, num_test = DataLoader('test', standardize=False)

dic = train_generator.class_indices
label_num = []

for label in test_label:
    label_num.append(dic[label])
    
for log in sorted(os.listdir(f'./logs/')):
    if log.startswith('batch16_0.87_ep021-'):
        model.load_weights(f'logs/{log}')
        print(log)
        print(np.count_nonzero(np.argmax(model.predict(test_img), axis=1) == label_num) / num_test)

batch16_0.87_ep021-loss0.017-auc0.9986570477485657.h5
0.9581895676882485
